### Tutorial Notebook: G<sub>0</sub>W<sub>0</sub> Approximation 

In this notebook, we present an example calculation of quasiparticle energies using QuatumMASALA's `gw` module.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Imports
import numpy as np
import sys

sys.path.append(".")

dirname = "./"

### DFT Calculation

We will start with a DFT calculation to get the energy eigenfunctions and eigenvalues.

In [3]:
import numpy as np

from qtm.constants import RYDBERG, ELECTRONVOLT
from qtm.kpts import KList
from qtm.lattice import RealLattice
from qtm.crystal import BasisAtoms, Crystal
from qtm.pseudo import UPFv2Data
from qtm.gspace import GSpace
from qtm.mpi import QTMComm
from qtm.dft import DFTCommMod, scf

from qtm.io_utils.dft_printers import print_scf_status

from qtm import qtmconfig
from qtm.logger import qtmlogger
# qtmconfig.fft_backend = 'mkl_fft'

from mpi4py.MPI import COMM_WORLD
comm_world = QTMComm(COMM_WORLD)
dftcomm = DFTCommMod(comm_world,1)

# Lattice
reallat = RealLattice.from_alat(alat=10.2,  # Bohr
                                a1=[-0.5,  0. ,  0.5],
                                a2=[ 0. ,  0.5,  0.5],
                                a3=[-0.5,  0.5,  0. ])

# Atom Basis
si_oncv = UPFv2Data.from_file('Si_ONCV_PBE-1.2.upf')
si_atoms = BasisAtoms.from_alat('Si', si_oncv, 28.086, reallat,
                               np.array([[0.875, 0.875, 0.875], [0.125, 0.125, 0.125]]))

crystal = Crystal(reallat, [si_atoms, ])  # Represents the crystal


# Generating k-points from a Monkhorst Pack grid (reduced to the crystal's IBZ)
mpgrid_shape = (4, 4, 4)
x = np.linspace(0,1,mpgrid_shape[0], endpoint=False)
y = np.linspace(0,1,mpgrid_shape[1], endpoint=False)
z = np.linspace(0,1,mpgrid_shape[2], endpoint=False)
xx,yy,zz = np.meshgrid(x,y,z, indexing="ij")
kcryst = np.vstack([xx.flatten(),yy.flatten(),zz.flatten()])
kpts = KList(recilat=crystal.recilat, k_coords=kcryst, k_weights=np.ones(kcryst.shape[1])/kcryst.shape[1])


# -----Setting up G-Space of calculation-----
ecut_wfn = 25 * RYDBERG
ecut_rho = 4 * ecut_wfn
grho = GSpace(crystal.recilat, ecut_rho)
gwfn = grho

# -----Spin-polarized (collinear) calculation-----
is_spin, is_noncolin = False, False
mag_start = None#[0.0]
numbnd_occ = 4
numbnd_nscf = 30

occ = 'fixed'

conv_thr = 1E-8 * RYDBERG
diago_thr_init = 1E-2 * RYDBERG

### DFT: SCF calculation for occupied bands

In [4]:
from qtm.kpts import KList


kpts = KList(recilat=crystal.recilat, k_coords=kpts.k_cryst, k_weights=np.ones(kpts.k_cryst.shape[1])/kpts.k_cryst.shape[1])

scf_out = scf(dftcomm, crystal, kpts, grho, gwfn,
          numbnd_occ, is_spin, is_noncolin,
          rho_start=mag_start, occ_typ=occ,
          conv_thr=conv_thr, diago_thr_init=diago_thr_init,
          iter_printer=print_scf_status,
          ret_vxc=True)


print("SCF Routine has exited")
# print(qtmlogger)

Iteration # 1, Run Time:   2.4 sec
Convergence Status   : NOT Converged
SCF Error           : 5.4518e-02 Ry
Avg Diago Iterations: 8.2
Diago Threshold     : 1.00e-02 Ry

Total Energy:          -15.93349186 Ry

      one-el:            4.87207970 Ry
     Hartree:            0.80642513 Ry
          XC:           -4.71223800 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            8.64406892 eV
----------------------------------------

Iteration # 1, Run Time:   3.1 sec
Convergence Status   : NOT Converged
SCF Error           : 6.0329e-02 Ry
Avg Diago Iterations: 2.8
Diago Threshold     : 6.81e-04 Ry

Total Energy:          -15.96523568 Ry

      one-el:            4.84033589 Ry
     Hartree:            0.80642513 Ry
          XC:           -4.71223800 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            5.90391780 eV
----------------------------------------

Iteration # 2, Run Time:   3.6 sec
Convergence Status   : NOT Converged
SCF Error           : 2.6122e-03 R

/home/agrim/codes/QuantumMASALA_October_merge_18Sept_branch_to_main/QuantumMASALA/src/qtm/dft/scf.py:420: ComplexWarning: Casting complex values to real discards the imaginary part
  vxc_arr[ik,iband] = hpsi_r


#### DFT: NSCF calculation for unshifted grid
Observe that `maxiter` has been set to `1` and `diago_thr_init` has been set to a high value.

In [6]:
rho = scf_out[1].copy()
nscf_out = scf(dftcomm, crystal, kpts, grho, gwfn,
          numbnd_nscf, is_spin, is_noncolin,
          rho_start=rho, 
          occ_typ=occ,
          conv_thr=conv_thr, 
          diago_thr_init=(conv_thr/crystal.numel)/10,
          iter_printer=print_scf_status,
          maxiter=1,
          ret_vxc=True)

scf_converged_nscf, rho_nscf, l_wfn_kgrp, en_nscf, vxc = nscf_out

Iteration # 1, Run Time:  22.5 sec
Convergence Status   :  Converged
SCF Error           : 8.9334e-10 Ry
Avg Diago Iterations: 20.0
Diago Threshold     : 1.25e-10 Ry

Total Energy:          -15.74866540 Ry

      one-el:            4.88767408 Ry
     Hartree:            1.10875920 Ry
          XC:           -4.84533999 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            6.47372136 eV
    LU Level:            7.11965854 eV
----------------------------------------



#### DFT: NSCF calculation for shifted grid

Dielectric matrix calculation for the $q\to 0$ point will require energy eigenfunctions for a slightly shifted $k$-grid.

In [7]:
k_coords_q = kpts.k_cryst+np.array([[0,0,0.001]]).T
k_weights_q = np.ones(k_coords_q.shape[1])/k_coords_q.shape[1]
kpts_q = KList(recilat=crystal.recilat, k_coords=k_coords_q, k_weights=k_weights_q)

rho = scf_out[1].copy()
out_q = scf(dftcomm, crystal, kpts_q, grho, gwfn,
          numbnd_nscf, is_spin, is_noncolin,
          rho_start=rho, 
          occ_typ=occ,
          conv_thr=conv_thr, 
          diago_thr_init=(conv_thr/crystal.numel)/10,
          iter_printer=print_scf_status,
          maxiter=1)

scf_converged_nscf_q, rho_nscf_q, l_wfn_kgrp_q, en_nscf_q = out_q


print("Shifted SCF Routine has exited")
# print(qtmlogger)

Iteration # 1, Run Time:  17.7 sec
Convergence Status   :  Converged
SCF Error           : 1.0736e-09 Ry
Avg Diago Iterations: 20.0
Diago Threshold     : 1.25e-10 Ry

Total Energy:          -15.74866457 Ry

      one-el:            4.88767491 Ry
     Hartree:            1.10875920 Ry
          XC:           -4.84533999 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            6.47369760 eV
    LU Level:            7.11794116 eV
----------------------------------------

Shifted SCF Routine has exited


### Load Input Files
Input data is handled by the ``EpsInp`` class.\
The data can be provided either by constructing the ``EpsInp`` object or by reading BGW-compatible input file ``epsilon.inp``.\
The attributes have been supplied with docstrings from BerkeleyGW's input specification, so they will be accessible directly in most IDEs.

In [8]:
from qtm.gw.io_bgw.epsinp import Epsinp

# Constructing input manually
# epsinp = Epsinp(epsilon_cutoff=1.2,
#                 use_wfn_hdf5=True,
#                 number_bands=8,
#                 write_vcoul=True,
#                 qpts=[[0.0,0.0,0.0]],
#                 is_q0=[True])

# Reading from epsilon.inp file
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')
# print(epsinp)

# There is an analogous system to read SigmaInp
from qtm.gw.io_bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')
# print(sigmainp)

### Initialize Epsilon Class

``Epsilon`` class can be initialized by either directly passing the required `quantummasala.core` objects or by passing the input objects discussed earlier.

In [9]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon
from qtm.klist import KList

kpts_gw =   KList(recilat=kpts.recilat,   cryst=kpts.k_cryst.T,   weights=kpts.k_weights)
kpts_gw_q = KList(recilat=kpts_q.recilat, cryst=kpts_q.k_cryst.T, weights=kpts_q.k_weights)

# Manual initialization
epsilon = Epsilon(
    crystal = crystal,
    gspace = grho,
    kpts = kpts_gw,
    kptsq = kpts_gw_q,        
    l_wfn = l_wfn_kgrp,
    l_wfnq = l_wfn_kgrp_q,
    qpts = QPoints.from_cryst(kpts.recilat, epsinp.is_q0, *epsinp.qpts),
    epsinp = epsinp,
)

# epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 16511.99it/s]


The three main steps involved in the calculation have been mapped to the corresponding functions:
1.  ``matrix_elements``: Calculation of Planewave Matrix elements
2.  ``polarizability``: Calculation of RPA polarizability matrix $P$
3.  ``epsilon_inverse``: Calculation of (static) epsilon-inverse matrix

<!-- 1.  ``matrix_elements``: Calculation of Planewave Matrix elements
    $$M_{nn'}({\textbf k},{\textbf q},{\textbf G}) = \bra{n\,{\textbf k}{+}{\textbf q}}e^{i({\textbf q}+{\textbf G})\cdot{\textbf r}}\ket{n'\,\textbf k}$$
    where the $\textbf G$-vectors included in the calculation satisfy $|\textbf q + \textbf G|^2 < E_{\text{cut}}$.
    Since this is a convolution in k-space, the time complexity can be reduced from $\mathcal{O}\left(N^2_G\right)$ to $\mathcal{O}\left(N_G\ln N_G\right)$ by using Fast Fourier Transform, where $N_G$  the number of reciprocal lattice vectors in the wavefunction.
    $$
    M_{nn'}({\bf k},{\bf q},\{{\bf G}\}) = {\rm FFT}^{-1}\left( \phi^{*}_{n,{\bf k}+{\bf q} }({\bf r}) \phi_{n',{\bf k} }({\bf r}) \right).
    $$
    where $\phi_{n',{\bf k}}({\bf r}) = {\rm FFT}\left( \psi_{n\bf k}(\bf G)\right)$. 
    
2.  ``polarizability``: Calculation of RPA polarizability matrix $P$
    $$
        P_{\textbf{GG'}}{\left({\textbf q}\;\!;0\right)}=
        \,\,{}\sum_{n}^{\textrm occ}\sum_{n'}^{\textrm emp}\sum_{{\textbf k}}
        \frac{
        \bra{n'\textbf k}e^{-i({\textbf q}+{\textbf G})\cdot{\textbf r}}\ket{n{\textbf k}{+}{\textbf q}}
        \bra{n{\textbf k}{+}{\textbf q}}e^{i({\textbf q}+{\textbf G'})\cdot{\textbf r}}\ket{n'\textbf k}
        }{E_{n{\textbf k}{+}{\textbf q}}\,{-}\,E_{n'{\textbf k}}}.
    $$
3.  ``epsilon_inverse``: Calculation of (static) epsilon-inverse matrix
    $$
        \epsilon_{\textbf{GG'}}{\left({\textbf q}\;\!\right)}=
        \delta_{\textbf{GG'}}\,{-}\,v{\left({\textbf q}{+}{\textbf G}\right)} \,
        P_{\textbf{GG'}}{\left({\textbf q}\;\!\right)}
    $$
    where $ v(\textbf{q} + \textbf{G}) = \frac{8\pi}{\left|\textbf q + \textbf G\right|^2} $ is bare Coulomb potential, written in Rydberg units. If this formula is used as-is for the case where $|\textbf q| = |\textbf G| = 0$, the resulting $v\left({\textbf{q=0}, \textbf{G=0}}\;\!\right)$ blows up as $1/q^2$. However, for 3D gapped systems, the matrix elements $\big| M_{nn'}\left({\bf k},{\textbf{q}\to\textbf{0}},{\textbf{G=0}}\right)\big| \sim q$ cancel the Coulomb divergence and $\epsilon_{\textbf{00}}\left({\textbf q\to\textbf{0}}\;\!\right) \sim q^2/q^2$ which is a finite number. In order to calculate $\epsilon_{\textbf{00}}\left({\textbf q=\textbf{0}}\;\!\right)$, we use the scheme specified in BGW2012, wherein $q=0$ is replaced with a small but non-zero value. Since matrix element calculation involves the eigenvectors $\ket{n{\textbf k}{+}{\textbf q}}$, having a non-$\Gamma$-centered $\textbf q\to 0$ point requires mean-field eigenvectors over a shifted $k$-grid. -->

In [10]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)
        
        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)


        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv

        if i_q == epsilon.qpts.index_q0:
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsmats.append(epsinv)
            
        if False:        
            # Compare the results with BGW's results
            if i_q == epsilon.qpts.index_q0:
                epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            else:
                epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])    

            # Calculate stddev between reference and calculated epsinv matrices
            std_eps = np.std(np.abs(epsref) - np.abs(epsinv)) / np.sqrt(np.prod(list(epsinv.shape)))
        
            epstol = 1e-16
            if np.abs(std_eps) > epstol:
                print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")
                print(np.max(np.abs(epsinv-epsref)))
                indices = np.where(np.abs(epsinv)-np.abs(epsref)>1e-5)
                

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)

calculate_epsilon()


Epsilon> q-pt index:   0%|          | 0/64 [00:00<?, ?it/s]

Epsilon> q-pt index: 100%|██████████| 64/64 [00:24<00:00,  2.61it/s]


### Sigma Calculation

Here we demonstate the calculation of diagonal matrix elements of $\Sigma_{\text{QP}}$. The input parameters for sigma calculation are being read from `sigma.inp` file, but the same parameters can also be provided by manually constructing a `SigmaInp` object. 

Here we will calculate $\bra{nk}\Sigma_{\text{QP}}\ket{nk}$ for the following k-points:
- $\Gamma$: `k=(0,0,0)`
- $L$: `k=(0.5,0.5,0)`
- $X$: `k=(0,0.5,0)`

In [11]:
from qtm.gw.sigma import Sigma

outdir = dirname+"temp/"


sigma = Sigma.from_qtm_scf(
    crystal = crystal,
    gspace = grho,
    kpts = kpts_gw,
    kptsq=kpts_gw_q,
    l_wfn_kgrp=l_wfn_kgrp,
    l_wfn_kgrp_q=l_wfn_kgrp_q,
    sigmainp=sigmainp,
    epsinp = epsinp,
    epsilon=epsilon,
    rho=rho,
    vxc=vxc
)

# Alternatively, the Sigma object can also be intitialized from pw2bgw.x output data (after being procesed by wfn2hdf5.x).
# sigma = Sigma.from_data(
#     wfndata=wfndata,
#     wfnqdata=wfnqdata,
#     sigmainp=sigmainp,
#     epsinp=epsinp,
#     l_epsmats=epsilon.l_epsinv,
#     rho=rho,
#     vxc=vxc,
#     outdir=outdir,
# )

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 12064.51it/s]


vcoul: Vcoul:
        * gspace = <qtm.gspace.gspc.GSpace object at 0x7fdbb82ae490>
        * qpts = <qtm.gw.core.QPoints object at 0x7fdb6bdc0910>
        * bare_coulomb_cutoff = 10.0
        * avgcut = 1e-05
        * l_gspace_q = <class 'list'> of length 64
        * vcoul = <class 'list'> of length 64
        * N_SAMPLES = 2500000.0
        * N_SAMPLES_COARSE = 250000.0
        * SEED = 5000
        


Vcoul calculation for qpts: 100%|██████████| 64/64 [00:49<00:00,  1.30it/s]


In [12]:
sigma_sx_cohsex_mat = sigma.sigma_sx_static(yielding=True)    
print("Sigma SX COHSEX")
sigma.pprint_sigma_mat(sigma_sx_cohsex_mat)

Sigma_SX_Static:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_SX_Static: 100%|██████████| 64/64 [00:02<00:00, 21.73it/s]

Sigma SX COHSEX
  n      ik=0        ik=40       ik=8    
  1    11.875832   10.670146   11.328755
  2     8.031250   10.680399   10.014675
  3     8.030519    8.719830    8.365721
  4     8.032419    8.719502    8.366188
  5     2.912191    2.586585    2.872630
  6     2.908211    2.588154    2.427566
  7     2.910656    1.328648    2.426289
  8     2.681673    1.329158    0.739615


In [13]:
sigma_ch_cohsex_mat = sigma.sigma_ch_static()    
print("Sigma CH COHSEX")
sigma.pprint_sigma_mat(sigma_ch_cohsex_mat)

Sigma_CH_Static_Partial:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:05<00:00, 12.52it/s]

Sigma CH COHSEX
  n      ik=0        ik=40       ik=8    
  1    -6.627249   -6.303086   -6.494450
  2    -5.099274   -6.303143   -6.068630
  3    -5.099179   -5.571198   -5.407134
  4    -5.099629   -5.571212   -5.407410
  5    -4.746872   -4.700738   -4.917365
  6    -4.745410   -4.701625   -4.287437
  7    -4.746343   -3.459957   -4.286988
  8    -4.757445   -3.460717   -2.975082


In [14]:
sigma.autosave=False
sigma.print_condition=True
cohsex_result = sigma.calculate_static_cohsex()

Sigma_X: 100%|██████████| 64/64 [00:02<00:00, 24.80it/s]


Sigma X
  n      ik=0        ik=40       ik=8    
  1   -17.549885  -16.038835  -16.918913
  2   -12.974838  -16.038768  -14.908717
  3   -12.974887  -13.417569  -13.204348
  4   -12.974943  -13.417522  -13.204514
  5    -5.765704   -5.196499   -5.934488
  6    -5.765349   -5.196223   -5.081719
  7    -5.765490   -3.789616   -5.081701
  8    -5.831181   -3.789609   -2.406364


Sigma_SX_Static: 100%|██████████| 64/64 [00:02<00:00, 24.31it/s]


Sigma SX STATIC
  n      ik=0        ik=40       ik=8    
  1    11.875832   10.670146   11.328755
  2     8.031250   10.680399   10.014675
  3     8.030519    8.719830    8.365721
  4     8.032419    8.719502    8.366188
  5     2.912191    2.586585    2.872630
  6     2.908211    2.588154    2.427566
  7     2.910656    1.328648    2.426289
  8     2.681673    1.329158    0.739615


Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:05<00:00, 12.27it/s]


Sigma CH STATIC
  n      ik=0        ik=40       ik=8    
  1    -6.627249   -6.303086   -6.494450
  2    -5.099274   -6.303143   -6.068630
  3    -5.099179   -5.571198   -5.407134
  4    -5.099629   -5.571212   -5.407410
  5    -4.746872   -4.700738   -4.917365
  6    -4.745410   -4.701625   -4.287437
  7    -4.746343   -3.459957   -4.286988
  8    -4.757445   -3.460717   -2.975082


Sigma_CH_Static_Exact: 100%|██████████| 64/64 [00:20<00:00,  3.07it/s]

Sigma CH EXACT STATIC
  n      ik=0        ik=40       ik=8    
  1    -7.444554   -7.623059   -7.626657
  2    -7.756865   -7.623276   -7.314291
  3    -7.756538   -7.477288   -7.674019
  4    -7.757690   -7.476949   -7.674193
  5    -7.335784   -6.795210   -7.354466
  6    -7.334351   -6.795007   -7.120593
  7    -7.335415   -7.673444   -7.120353
  8    -7.829756   -7.674547   -6.232165
Sig (Exact):
  n      ik=0        ik=40       ik=8    
  1   -13.118608  -12.991748  -13.216814
  2   -12.700453  -12.981645  -12.208332
  3   -12.700906  -12.175028  -12.512646
  4   -12.700214  -12.174969  -12.512519
  5   -10.189297   -9.405124  -10.416324
  6   -10.191490   -9.403076   -9.774746
  7   -10.190250  -10.134413   -9.775765
  8   -10.979265  -10.134998   -7.898913
Eqp0 (Exact):
  n      ik=0        ik=40       ik=8    
  1    -8.251716   -3.521709   -5.575061
  2     5.142703   -3.511605   -2.586259
  3     5.142250    2.014979    3.845956
  4     5.142942    2.015038    3.846082
  5  

In [15]:
sigma.print_condition=True
print("Sigma CH COHSEX EXACT")
sigma_ch_exact_mat = sigma.sigma_ch_static_exact()    
sigma.pprint_sigma_mat(sigma_ch_exact_mat)

Sigma CH COHSEX EXACT


Sigma_CH_Static_Exact:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_CH_Static_Exact: 100%|██████████| 64/64 [00:22<00:00,  2.91it/s]

  n      ik=0        ik=40       ik=8    
  1    -7.444554   -7.623059   -7.626657
  2    -7.756865   -7.623276   -7.314291
  3    -7.756538   -7.477288   -7.674019
  4    -7.757690   -7.476949   -7.674193
  5    -7.335784   -6.795210   -7.354466
  6    -7.334351   -6.795007   -7.120593
  7    -7.335415   -7.673444   -7.120353
  8    -7.829756   -7.674547   -6.232165


In [16]:
sigma.print_condition=False
sigma_sx_gpp = sigma.sigma_sx_gpp()    
print("Sigma SX GPP")
sigma.pprint_sigma_mat(sigma_sx_gpp)

Sigma_SX_GPP: 100%|██████████| 64/64 [00:50<00:00,  1.27it/s]

Sigma SX GPP
  n      ik=0        ik=40       ik=8    
  1    12.243998   10.836142   11.549630
  2     8.177246   10.848005   10.174679
  3     8.176874    8.838429    8.500881
  4     8.179714    8.837812    8.501443
  5     3.300257    2.864940    3.299234
  6     3.291178    2.867032    2.791634
  7     3.296973    1.736864    2.790402
  8     3.369051    1.738987    1.089915


In [17]:
sigma.print_condition=False
sigma_ch_gpp,_ = sigma.sigma_ch_gpp()    
print("Sigma CH GPP")
sigma.pprint_sigma_mat(sigma_ch_gpp)

Sigma_CH_GPP:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_CH_GPP: 100%|██████████| 64/64 [00:59<00:00,  1.08it/s]

Sigma CH GPP
  n      ik=0        ik=40       ik=8    
  1    -5.796970   -5.846687   -5.879218
  2    -5.402481   -5.849324   -5.642409
  3    -5.402850   -5.559384   -5.572965
  4    -5.404011   -5.559113   -5.573327
  5    -5.343540   -4.982179   -5.446976
  6    -5.336222   -4.983590   -4.890853
  7    -5.341045   -4.621046   -4.890394
  8    -5.721550   -4.622911   -3.771960


In [18]:
gpp_result = sigma.calculate_gpp()

Sigma_X: 100%|██████████| 64/64 [00:02<00:00, 21.82it/s]

Sigma X GPP


  n      ik=0        ik=40       ik=8    
  1   -17.549885  -16.038835  -16.918913
  2   -12.974838  -16.038768  -14.908717
  3   -12.974887  -13.417569  -13.204348
  4   -12.974943  -13.417522  -13.204514
  5    -5.765704   -5.196499   -5.934488
  6    -5.765349   -5.196223   -5.081719
  7    -5.765490   -3.789616   -5.081701
  8    -5.831181   -3.789609   -2.406364


Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:05<00:00, 12.49it/s]

Sigma CH STATIC COHSEX


...

Sigma_CH_GPP: 100%|██████████| 64/64 [01:02<00:00,  1.02it/s]

Sigma CH GPP
  n      ik=0        ik=40       ik=8    
  1    -5.796970   -5.846687   -5.879218
  2    -5.402481   -5.849324   -5.642409
  3    -5.402850   -5.559384   -5.572965
  4    -5.404011   -5.559113   -5.573327
  5    -5.343540   -4.982179   -5.446976
  6    -5.336222   -4.983590   -4.890853
  7    -5.341045   -4.621046   -4.890394
  8    -5.721550   -4.622911   -3.771960
Sig GPP:
  n      ik=0        ik=40       ik=8    
  1   -11.511509  -11.709366  -11.814604
  2   -11.528869  -11.700154  -10.999277
  3   -11.529542  -11.091570  -11.409874
  4   -11.528270  -11.091692  -11.409789
  5    -9.103443   -8.360974   -9.300780
  6    -9.104864   -8.359472   -8.597516
  7    -9.104099   -8.780541   -8.598375
  8    -9.719836   -8.780448   -6.716951
Eqp0


  n      ik=0        ik=40       ik=8    
  1    -6.644618   -2.239326   -4.172851
  2     6.314287   -2.230114   -1.377204
  3     6.313614    3.098436    4.948728
  4     6.314886    3.098314    4.948813
  5     9.984195    7.766294    8.876921
  6     9.982774    7.767796   10.871420
  7     9.983540   18.518438   10.870562
  8    11.226073   18.518532   15.216428


Sigma_SX_GPP: 100%|██████████| 64/64 [00:52<00:00,  1.22it/s]

Sigma CH GPP dE
  n      ik=0        ik=40       ik=8    
  1    -6.056253   -6.123548   -6.149623
  2    -5.641589   -6.126418   -5.916338
  3    -5.637486   -5.843677   -5.860942
  4    -5.638679   -5.843399   -5.861216
  5    -5.570835   -5.285448   -5.772163
  6    -5.562631   -5.288282   -5.100439
  7    -5.567930   -4.907922   -5.099101
  8    -5.941014   -4.910020   -4.063206
Sigma SX GPP dE
  n      ik=0        ik=40       ik=8    
  1    -6.056253   -6.123548   -6.149623
  2    -5.641589   -6.126418   -5.916338
  3    -5.637486   -5.843677   -5.860942
  4    -5.638679   -5.843399   -5.861216
  5    -5.570835   -5.285448   -5.772163
  6    -5.562631   -5.288282   -5.100439
  7    -5.567930   -4.907922   -5.099101
  8    -5.941014   -4.910020   -4.063206
Z:
  n      ik=0        ik=40       ik=8    
  1     0.714882    0.775219    0.755521
  2     0.839971    0.775067    0.778936
  3     0.839967    0.817147    0.829068
  4     0.839961    0.817134    0.829062
  5     0.830705   